# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.131046e+02     1.582444e+00
 * time: 0.08885979652404785
     1     1.058032e+01     8.903073e-01
 * time: 0.9146888256072998
     2    -1.156324e+01     1.060962e+00
 * time: 1.0458118915557861
     3    -3.416504e+01     7.439072e-01
 * time: 1.2106029987335205
     4    -4.767007e+01     6.061275e-01
 * time: 1.3653128147125244
     5    -5.687900e+01     2.158718e-01
 * time: 1.5126619338989258
     6    -5.977996e+01     1.444664e-01
 * time: 1.6232099533081055
     7    -6.094448e+01     5.073819e-02
 * time: 1.7326748371124268
     8    -6.138321e+01     6.294586e-02
 * time: 1.8480658531188965
     9    -6.165207e+01     4.435873e-02
 * time: 1.9576029777526855
    10    -6.185917e+01     3.120079e-02
 * time: 2.076235771179199
    11    -6.201414e+01     2.402019e-02
 * time: 2.18849778175354
    12    -6.207919e+01     1.906550e-02
 * time: 2.306353807449341
    13    -6.213801e+01     1.472747e-02
 * time: 2.4196629524230

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557706
    AtomicNonlocal      14.8522672
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336821

    total               -62.261666460849
